# Performing Testing at Each Round of Training 

Use for developing (autoreloads changes made across packages)

In [ ]:
%load_ext autoreload
%autoreload 2

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Define an experiment model and parameters"

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [ ]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [ ]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100, # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
    'test_ratio': .3,
    'test_on_local_updates': True, 
    'test_on_global_updates': True
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [ ]:
exp.run()

In [ ]:
exp.run(rounds=1 , increase=True)

In [ ]:
exp.run(rounds=8, increase=True)

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()

Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())


Feel free to run other sample notebooks or try your own models :D

## Testing using your own testing metric

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlanCM(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlanCM, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss

    def testing_step(self, data, target):
        
        output = self.forward(data)
        loss1   = torch.nn.functional.nll_loss(output, target)
        output = self(data)
        loss2   = torch.nn.functional.nll_loss(output, target)
        return [loss1, loss2]

In [2]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100, # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
    'test_ratio': .3,
    'test_on_local_updates': True, 
    'test_on_global_updates': True
}

In [3]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlanCM,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-03-25 14:25:26,830 fedbiomed INFO - Component environment:
2022-03-25 14:25:26,832 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-03-25 14:25:27,421 fedbiomed INFO - Messaging researcher_9da5c1c0-e6d2-43e5-9191-ce676c5aac98 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f592c8c26d0>
2022-03-25 14:25:27,460 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-03-25 14:25:37,489 fedbiomed INFO - Node selected for training -> node_409df974-a78a-4fdd-9f09-eac2963ff402
2022-03-25 14:25:37,598 fedbiomed DEBUG - Model file has been saved: /home/scansiz/Desktop/Inria/development/fedbiomed/var/experiments/Experiment_0018/my_model_d5a68b29-96f1-4832-a2e0-f23b94dc9bc5.py
2022-03-25 14:25:37,648 fedbiomed DEBUG - upload (HTTP POST request) of file /home/scansiz/Desktop/Inria/development/fedbiomed/var/experiments/Experiment_0018/my_model_d5a68b29-96f1-4832-a2e0-f23b94dc9bc5.py successful, 

In [4]:
exp.run()

2022-03-25 14:25:38,072 fedbiomed INFO - Sampled nodes in round 0 ['node_409df974-a78a-4fdd-9f09-eac2963ff402']
2022-03-25 14:25:38,075 fedbiomed INFO - Sending request 
					 To: node_409df974-a78a-4fdd-9f09-eac2963ff402 
					 Reqeust: : Perform training with the arguments: {'researcher_id': 'researcher_9da5c1c0-e6d2-43e5-9191-ce676c5aac98', 'job_id': '4d96f0c1-3ccd-4e04-8014-9f5be25d78ee', 'training_args': {'test_ratio': 0.3, 'test_on_local_updates': True, 'test_on_global_updates': True, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/25/my_model_d5a68b29-96f1-4832-a2e0-f23b94dc9bc5.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/25/aggregated_params_init_27b09b9c-31db-40d9-abb0-b40180005cf0.pt', 'model_class': 'MyTrainingPlanCM', 'training_data': {'node_409df974-a78a-4fdd-9f

2022-03-25 14:27:00,967 fedbiomed INFO - TRAINING 
					 NODE_ID: node_409df974-a78a-4fdd-9f09-eac2963ff402 
					 Epoch: 1 | Completed: 1440/42000 (3%) 
 					 Loss: 0.287531 
					 ---------
2022-03-25 14:27:01,672 fedbiomed INFO - TRAINING 
					 NODE_ID: node_409df974-a78a-4fdd-9f09-eac2963ff402 
					 Epoch: 1 | Completed: 1920/42000 (5%) 
 					 Loss: 0.442801 
					 ---------
2022-03-25 14:27:02,296 fedbiomed INFO - TRAINING 
					 NODE_ID: node_409df974-a78a-4fdd-9f09-eac2963ff402 
					 Epoch: 1 | Completed: 2400/42000 (6%) 
 					 Loss: 0.268874 
					 ---------
2022-03-25 14:27:02,968 fedbiomed INFO - TRAINING 
					 NODE_ID: node_409df974-a78a-4fdd-9f09-eac2963ff402 
					 Epoch: 1 | Completed: 2880/42000 (7%) 
 					 Loss: 0.107753 
					 ---------
2022-03-25 14:27:03,611 fedbiomed INFO - TRAINING 
					 NODE_ID: node_409df974-a78a-4fdd-9f09-eac2963ff402 
					 Epoch: 1 | Completed: 3360/42000 (8%) 
 					 Loss: 0.125760 
					 ---------
2022-03-25 14:27:04,248 fedbiomed I

2

2022-03-25 15:11:51,128 fedbiomed INFO - ERROR
					 NODE test_logger_node_97fcc681-0a4f-4597-a807-951d767494fa
					 MESSAGE: mqtt+console ERROR message
-----------------------------------------------------------------
2022-03-25 15:11:59,218 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Messaging mock_researcher_XXX successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7fb0e94462b0>
-----------------------------------------------------------------
2022-03-25 15:12:01,273 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Actual/True values (y_true) has more than two levels, using multiclass `weighted` calculation for the metric F1_SCORE
-----------------------------------------------------------------
2022-03-25 15:12:01,275 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Actual/True values (y_true) has more than two levels, using multiclass `weighted` calculation for the metric RECALL
-------------------

2022-03-25 15:12:02,496 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Recreating hashing for : monai-image-classification.txt 	 model_c0d3c19e-b25b-4010-9ebe-d11d933049bc
-----------------------------------------------------------------
2022-03-25 15:12:02,539 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Recreating hashing for : sklearn-sgdregressor.txt 	 model_9e33626f-c51c-4317-9d5a-030823d6ec4b
-----------------------------------------------------------------
2022-03-25 15:12:02,544 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: {'name': 'pytorch-celeba.txt', 'description': 'Default model', 'hash': 'de7355b328bf9d23aca667b86e0955c439eed583e0f9304b9df42e0a69776a1798f314273974ba7ac1b05220007d43aa', 'model_path': '/tmp/_nod_/default_models/pytorch-celeba.txt', 'model_id': 'model_c62acd1f-36f0-4f38-bcce-9f4f0926aed7', 'model_type': 'default', 'algorithm': 'SHA384', 'date_created': '25-03-2022 15:11:36.844704', 'date_modified': '25-03-2022 15:11:36.844

2022-03-25 15:12:02,979 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Recreating hashing for : pytorch-celeba.txt 	 model_c62acd1f-36f0-4f38-bcce-9f4f0926aed7
-----------------------------------------------------------------
2022-03-25 15:12:03,000 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Recreating hashing for : pytorch-mnist.txt 	 model_81e82839-f2cd-4921-b96a-89e60c7cddb6
-----------------------------------------------------------------
2022-03-25 15:12:03,020 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Recreating hashing for : pytorch-csv.txt 	 model_46b50240-d852-408c-a759-3fc345a18032
-----------------------------------------------------------------
2022-03-25 15:12:03,028 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Recreating hashing for : variational-autoencoder.txt 	 model_216b08c1-d96c-46f3-add7-36fe786f4f1b
-----------------------------------------------------------------
2022-03-25 15:12:03,049 fedbiomed INFO - INFO
	

2022-03-25 15:12:03,507 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Recreating hashing for : pytorch-usedcars.txt 	 model_16471146-c5ec-4d2f-bda0-a4a995e59042
-----------------------------------------------------------------
2022-03-25 15:12:03,520 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Recreating hashing for : monai-image-registration.txt 	 model_ac9c727a-232c-49eb-a3fb-5fe4615ed878
-----------------------------------------------------------------
2022-03-25 15:12:03,545 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Recreating hashing for : pytorch-celaba.txt 	 model_19dfdef2-54fd-4851-bff2-1c76931790d1
-----------------------------------------------------------------
2022-03-25 15:12:03,569 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Recreating hashing for : sklearn-perceptron.txt 	 model_e5274aa0-9b89-4038-aef6-6b1c54ea8b59
-----------------------------------------------------------------
2022-03-25 15:12:03,574 fedbiomed IN

2022-03-25 15:12:04,397 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Modified default model file has been detected. Hashing will be updated for: sklearn-perceptron.txt
-----------------------------------------------------------------
2022-03-25 15:12:04,404 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Modified default model file has been detected. Hashing will be updated for: variational-autoencoder.txt
-----------------------------------------------------------------
2022-03-25 15:12:04,427 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Modified default model file has been detected. Hashing will be updated for: pytorch-celaba.txt
-----------------------------------------------------------------
2022-03-25 15:12:04,453 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Modified default model file has been detected. Hashing will be updated for: pytorch-csv.txt
-----------------------------------------------------------------
2022-03-25 15:12:0

2022-03-25 15:12:06,094 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: Undetermined error during the testing phase on global parameter updates: 'FakeModel' object has no attribute 'testing_routine'
-----------------------------------------------------------------
2022-03-25 15:12:07,096 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: Undetermined error during the testing phase on local parameter updates'FakeModel' object has no attribute 'testing_routine'
-----------------------------------------------------------------
2022-03-25 15:12:07,096 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: Undetermined error during the testing phase on global parameter updates: 'FakeModel' object has no attribute 'testing_routine'
-----------------------------------------------------------------
2022-03-25 15:12:07,104 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: Undetermined error during the testing phase on local parameter updates'FakeModel' object has